In [177]:
import os
import sys
import json
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from loguru import logger as log
from typing import Union
from dtaidistance import dtw_visualisation as dtwvis

In [178]:
# Custom packages
from sediment_dtw import *
from utils import convert_warp_path_to_timeseries
from plotting import create_graph

In [179]:
log.remove()
log.add(sys.stderr, level="DEBUG");

In [180]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Define initial parameters

For example, if biostratigraphic data indicates that the age of oldest sediments in the core cannot exceed 245,000 years, set the `max_age` variable to 245. Similarly, set the `min_age` variable to the minimum age you expect the core top to be. For piston core from the ocean bottom, it is a good idea to set this to 0, but if data is available, such as for example the topmost 10k years are missing, this variable can be set to start at something else than 0. 

In [181]:
min_age = 71  # in kiloyears (kyrs) before present
max_age = 600 # in kiloyears (kyrs) before present
time_step = 5  # in kiloyears

### Find optimal distance between two time-series

#### Run parameters

Define the name, location and variable (column) names of the target/reference curve here.

In [182]:
ref = "LR04stack"
# ref = "core_1100_conventional_agemodel"
# ref = "core_1100_dtw_agemodel"
ref_path = f'../data/{ref}.csv'
ref_cols = ['time', 'd18O']

#### Load target (reference) data and limit by `min_age` and `max_age`

Load target data, and filter by expected minimum and maximum age (`min_age`, `max_age`) 

In [183]:
target = pd.read_csv(ref_path, usecols=ref_cols) 
target = target[target['time'] <= max_age]
target = target[target['time'] >= min_age]

The next step is only executed if the dtw_age model for this specific core is to be used, in this case 1100. This age model was created in a previous iteration of this method.

In [184]:
if ref == "core_1100_dtw_agemodel":
    log.debug("Resampling reference time curve")
    target = pd.DataFrame(target.groupby('time', as_index=False)['d18O'].median())

In [185]:
target = target.reset_index(drop=True)

In [186]:
target.head(2)

,time,d18O
0,71.0,4.22
1,72.0,4.21


#### Load data curve (unknown time interval)

In [187]:
# source = "../data/core_1100_d18O_pl.csv"
# source = "../data/core_1150_d18O_pl.csv"
# source = "../data/core_1150_d18O_split_1.csv"
# source = "../data/core_1150_aragonite_split_1.csv"
source = "../data/core_1150_d18O_bulk_split_1.csv" 

In [188]:
source_name = source.split("/")
source_name = source_name[-1].split("_")[:2]
source_name = "_".join(source_name)

source_var = source.split("/")
source_var = source_var[-1].split("_")[2:]
source_var = "_".join(source_var)
source_var = source_var.split(".")[0]

data = pd.read_csv(source, skip_blank_lines=True)
data.head(2)

,depth_m,d18O_bulk
0,2.06,-1.136975
1,2.11,-1.552736


In [189]:
source_name

'core_1150'

In [190]:
source_var

'd18O_bulk_split_1'

In [191]:
data_col = data.columns[1]
data_col

'd18O_bulk'

#### Define DTW object

In [192]:
dtw = SedimentDTW(target=target, 
                  data=data, 
                  normalize=True, 
                  smooth=True, 
                  window_size=11, 
                  polynomial=3)

2023-05-04 22:30:49.441 | DEBUG    | sediment_dtw:__init__:91 - Time-warp object created successfully!


##### Find simple distance

In [193]:
simple_distance = dtw.simple_distance()
print(round(simple_distance, 2))

14.73


##### Find minimum distance iteratively

In [194]:
distance, time, min_distances = dtw.find_min_distance(min_age, max_age, time_step, warp_path=True)
print(f'Found minimum distance: {round(distance, 2)} at target time {time} kyrs')

2023-05-04 22:30:54.330 | DEBUG    | sediment_dtw:find_min_distance:194 - 5.540740958460823, 5.540740958460823


Found minimum distance: 5.54 at target time 261 kyrs


##### Save min_distances to file

In [195]:
_tmp_df = pd.DataFrame(min_distances.keys(),min_distances.values()).reset_index(drop=False).rename(columns={'index': 'distance', 0: 'time'})
_tmp_df.to_csv(f"../out/core_1150_min_distances_{source_var}_ref-{ref}.csv", index=False)

#### Create age models

In [196]:
def select_all_pairs(warp_path, index):
    pairs = list()
    for pair in warp_path:
        if pair[0] == index:
            pairs.append(pair)
    return pairs

In [197]:
def select_max_from_pairs(pairs):
    values = list()
    for pair in pairs:
        values.append(pair[1])
    return max(values)

In [198]:
def convert_warp_path_to_timeseries(target: Union[list, np.array, pd.Series], 
                                    data: Union[list, np.array, pd.Series],
                                    warp_path: list[tuple]):
    
    dtw_time_axis = list()
    for index, _ in enumerate(data):
        pairs = select_all_pairs(warp_path, index)
        maximum = select_max_from_pairs(pairs)
        dtw_time_axis.append(target[maximum])

    assert len(dtw_time_axis) == len(data)
    return dtw_time_axis

In [199]:
dtw_time_axis = convert_warp_path_to_timeseries(target['time'], data['depth_m'], dtw.best_path)

In [200]:
data['time'] = dtw_time_axis

In [201]:
data.head()

,depth_m,d18O_bulk,time
0,2.06,-1.136975,71.0
1,2.11,-1.552736,105.0
2,2.21,-0.923401,112.0
3,2.26,-1.603000,113.0
4,2.31,-1.509563,114.0


In [202]:
print(data_col, ref, source_name, source_var)

d18O_bulk LR04stack core_1150 d18O_bulk_split_1


In [203]:
data.to_csv(f"../out/{source_name}_{source_var}_{ref}_out.csv", index=False)

#### Combine conventional and DTW age model if required

In [204]:
data.head(2)

,depth_m,d18O_bulk,time
0,2.06,-1.136975,71.0
1,2.11,-1.552736,105.0


In [205]:
=

SyntaxError: invalid syntax (1763773627.py, line 1)

In [ ]:
_tmp = pd.read_csv("../out/time-depth-models/tdm_core_1150_d18O_LR04stack_out.csv")

In [ ]:
_tmp.head(5)

In [ ]:
_tmp_2 = pd.read_csv("../data/core_1150_aragonite_split_0.csv")

In [ ]:
_tmp_2['time'] = _tmp_2['depth_m'].map(dict(zip(_tmp['depth_m'], _tmp['time'])))

In [ ]:
_tmp_2

In [ ]:
time_depth_model = pd.concat([_tmp_2, data]).reset_index(drop=True)

In [ ]:
time_depth_model

In [ ]:
time_depth_model = time_depth_model[['depth_m','time', 'aragonite']]

In [ ]:
var = source_var.split("_")[0]
time_depth_model.to_csv(f"../out/time-depth-models/tdm_{source_name}_{var}_{ref}_out.csv", index=False)